In [ ]:
# clean up the data
!sed -n -r '/[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}:[^\s]+/p' data/combined.txt > data/comb_out.txt

In [ ]:
# convert to csv
!sed 's/:/,/' data/comb_out.txt > data/comb_out.csv

In [2]:
# total count
!wc -l data/comb_out.csv

2687437913 data/comb_out.csv


In [ ]:
!python3 reservoir_sampling.py data/comb_out.csv 1000000 > data/comb_1m.csv

In [ ]:
## reservoir_sampling.py
import sys
import random

filename = sys.argv[1]
sample_size = int(sys.argv[2])
reservoir = []

with open(filename, 'r') as f:
    for i, line in enumerate(f):
        if i < sample_size:
            reservoir.append(line)
        else:
            r = random.randint(0, i)
            if r < sample_size:
                reservoir[r] = line

for line in reservoir:
    print(line, end='')

In [ ]:
!sed 's/,\(.*$\)/,"\1"/g' comb_1m.csv > comb_1m_out.csv

Average record size = Total file size / Number of records <br>
                      = 82 GB / 2,687,437,913 records <br>
                      = approximately 31.5 bytes/record <br>

Number of records for 3 GB ≈ 3 GB / average record size <br>
                           ≈ (3 * 1024 * 1024 * 1024) bytes / 31.5 bytes/record <br>
                           ≈ 102,236,404 records <br>

In [1]:
import multiprocessing
import random
import os

# Define the number of records to sample
num_records_to_sample = 102236404 * 3
total_records = 2687437913
record_size = 31.5  # This should be adjusted to your actual average record size

# Calculate number of records per chunk for parallel processing
num_processes = multiprocessing.cpu_count()  # Number of processes to create
chunk_size = total_records // num_processes

# Function to process each chunk and select records
def process_chunk(chunk_index, return_dict):
    # Calculate the start and end index of the chunk
    start_index = chunk_index * chunk_size
    end_index = start_index + chunk_size if chunk_index < num_processes - 1 else total_records
    chunk_records_to_sample = num_records_to_sample // num_processes

    selected_records = set(random.sample(range(start_index, end_index), chunk_records_to_sample))

    selected_data = []
    with open('data/comb_out.csv', 'r') as file:
        file.seek(int(start_index * record_size))  # Seek to the start of the chunk
        for i in range(start_index, end_index):
            record = file.readline()
            if i in selected_records:
                selected_data.append(record)
            if not record:  # End of file
                break

    # Store result in return dictionary
    return_dict[chunk_index] = selected_data

# Create a manager to hold the returned data from each process
manager = multiprocessing.Manager()
return_dict = manager.dict()

# Create a list to keep track of processes
processes = []

# Create and start processes
for i in range(num_processes):
    p = multiprocessing.Process(target=process_chunk, args=(i, return_dict))
    processes.append(p)
    p.start()

# Ensure all processes have finished execution
for p in processes:
    p.join()

# Combine the data from all processes
combined_data = []
for chunk_data in return_dict.values():
    combined_data.extend(chunk_data)

# Now `combined_data` holds the randomly selected records

# Write the combined data to the output file
with open('data/comb_9g.csv', 'w') as outfile:
    outfile.writelines(combined_data)

In [4]:
!sed 's/,\(.*$\)/,"\1"/g' data/comb_9g.csv > data/comb_9g_out.csv